In [2]:
import fitz
import re
from datetime import datetime
import pandas as pd


In [3]:

# Load the Excel file (replace with the actual file path)
file_path = 'Mario_Gutierrez_BR.xlsx'
sheet_name = 'CPT DESC'  # Your sheet name

# Read the Excel file into a DataFrame
df_cpt = pd.read_excel(file_path, sheet_name=sheet_name, dtype={'Procedure Code': str, 'Description': str})


In [4]:
df_cpt.head()

,Procedure Code,Description
0,95004,Percut allergy s
1,95010,PERCUT ALLERGY T
2,95012,Exhaled nitric o
3,95015,ID ALLERGY TITRA
4,95017,Perq & icut allg


### Hospital Name extraction

In [5]:
my_path = r'R000548 - R000550 M. Gutierrez_EMG Center Of Houston_Billing.pdf'
doc = fitz.open(my_path)

In [6]:
page = doc[0]
all_infos = page.get_text(sort=True)
print(all_infos)

      PERTAIN TO :    Mario Alberto Gutierrez

         FROM :  EMG Center of Houston
                          P.o. Box 540905
                   HOUSTON, TEXAS 77254



        PHONE :    (713) 622-7744          FAX :   (832) 202-2728

   RECORD TYPE :     Billing


      DELIVER TO :    Trent Shelton

                 ARNOLD & ITKIN, LLP
                      6009 MEMORIAL DRIVE
                  HOUSTON, TEXAS 77007


        PHONE :    (713) 222-3800          FAX :   (713) 222-3850





Order No : 225590-1-22

                                                 MARIO GUTIERREZ
                                                                R000548


In [7]:
def extract_hospital_name(text):

    match = re.search(r"FROM\s*:\s*([^\n]+)", text)
    if match:
        return match.group(1).strip() 
    return "Unknown Hospital"

In [8]:
extract_hospital_name(all_infos)

'EMG Center of Houston'

### Hospital Billing Extraction

In [9]:
page = doc[2]
all_infos = page.get_text(sort=True)
print(all_infos)

                                  EMG Center of Houston
                                    Itemization of Charges




Clinic:          EMG Center of Houston

Address:            4151 Southwest FWY                       Insurance #1:          Self Pay-Non Ins
                    410                                                        4151 Southwest Fwy Ste 410
                      Houston, TX 770277320                                            Houston, TX 770277320


Phone:                (713) 622-7744                               Adjuster:

Tax ID:              463385579                            Group Number

WCAB:                                                           Policy Number:      24560


Employer:                                                     Insurance #2:

Patient #:           4090                                        Adjuster:

Patient:           MARIO GUTIERREZ                     Group Number:

Date of Injury:        06/26/2023                         

In [32]:
# Regular expression pattern to match the relevant billing data
pattern = r'(\d{4})\s+(\d{2}/\d{2}/\d{4})\s+(.+?)\s+([A-Z0-9]+)\s+([\d,]+\.\d{2})'
# Find all matches in the text
matches = re.findall(pattern, all_infos)

In [33]:
matches

[('4180', '04/19/2024', 'Needle measurement of elec...', '95861', '1,300.00'),
 ('4180', '04/19/2024', 'Needle measurement of elec...', '95886', '1,100.00'),
 ('4180', '04/19/2024', 'Nerve conduction, 11-12 st...', '95912', '950.00'),
 ('4180', '04/19/2024', 'OFFICE OR OTHER OUTPATIENT...', '99203', '375.00'),
 ('4180', '04/19/2024', 'Sterile needle', 'A4215', '100.00'),
 ('4180', '04/19/2024', 'Conductive gel or paste', 'A4558', '50.00'),
 ('4181', '04/19/2024', 'Needle measurement of elec...', '95861', '1,300.00'),
 ('4181', '04/19/2024', 'Needle measurement of elec...', '95886', '1,100.00'),
 ('4181', '04/19/2024', 'Nerve conduction, 7-8 stud...', '95910', '950.00'),
 ('4181', '04/19/2024', 'Sterile needle', 'A4215', '100.00'),
 ('4181', '04/19/2024', 'Electrodes, pair', 'A4556', '75.00'),
 ('4181', '04/19/2024', 'Conductive gel or paste', 'A4558', '50.00')]

In [10]:
def extract_billing_info_pdf(file_path):
    # Open the PDF file
    doc = fitz.open(file_path)
    billing_records = []

    # Extract hospital name from first page
    
    first_page_text = doc[0].get_text(sort=True)
    hospital_name = extract_hospital_name(first_page_text)
    
    # Loop through each page of the PDF
    for page in doc:
        # Extract the text from the page
        all_medicine = page.get_text(sort=True)
        
        # Regular expression pattern to match the relevant billing data
        pattern = r'(\d{4})\s+(\d{2}/\d{2}/\d{4})\s+(.+?)\s+([A-Z0-9]+)\s+([\d,]+\.\d{2})'
        # Find all matches in the text
        matches = re.findall(pattern, all_medicine)

        # Loop through each match and create a record
        for match in matches:
            record = {
                "Hospital": hospital_name,  # Add hospital name here
                "Date": datetime.strptime(match[1], '%m/%d/%Y'),  # Convert to datetime object
                "CPT": str(match[3]),  # Convert to string
                "Extracted_Description": str(match[2].strip()),  # Convert to string and strip any extra whitespace
                "Charges": float(match[4].replace(',', ''))  # Remove commas and convert to float
            }
            billing_records.append(record)  # Append each record to the list

    # Create a DataFrame from the list of records
    df_billing_records = pd.DataFrame(billing_records)

    return df_billing_records  # Return the DataFrame

In [11]:
df_extracted = extract_billing_info_pdf(my_path)

In [12]:
df_cpt.columns

Index(['Procedure Code', 'Description'], dtype='object')

In [13]:
# Matching columns CPT with Procedure Code


final_data = pd.merge(df_extracted,df_cpt,left_on='CPT', right_on='Procedure Code', how='left')
final_data.drop(columns=['Procedure Code'], inplace=True)
final_data.rename(columns={'Description': 'Provided_Description'}, inplace=True)
final_data.head()

,Hospital,Date,CPT,Extracted_Description,Charges,Provided_Description
0,EMG Center of Houston,2024-04-19,95861,Needle measurement of elec...,1300.0,Muscle test 2 li
1,EMG Center of Houston,2024-04-19,95886,Needle measurement of elec...,1100.0,Musc test w/n
2,EMG Center of Houston,2024-04-19,95912,"Nerve conduction, 11-12 st...",950.0,Nrv cndj test 11
3,EMG Center of Houston,2024-04-19,99203,OFFICE OR OTHER OUTPATIENT...,375.0,Office Visit
4,EMG Center of Houston,2024-04-19,A4215,Sterile needle,100.0,"Needle, sterile"


In [14]:
# Biling Charges Matching

final_data['Charges'].sum()

7450.0

### Convert to CSV

In [15]:
# final_data.to_csv('R000548- R000550.csv')